In [ ]:
import math
import numpy as np
from numpy.random import seed
import random
import pandas as pd

import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

import tensorflow as tf
import tensorflow.keras.utils as Utils
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras import utils as np_utils

In [ ]:
# Seed Everything to reproduce results for future use cases
def seed_everything(seed=42):
    # Seed value for TensorFlow
    tf.random.set_seed(seed)
    
    # Seed value for NumPy
    np.random.seed(seed)
    
    # Seed value for Python's random library
    random.seed(seed)
    
    # Force TensorFlow to use single thread
    # Multiple threads are a potential source of non-reproducible results.
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )

    # Make sure that TensorFlow uses a deterministic operation wherever possible
    tf.compat.v1.set_random_seed(seed)

    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

seed_everything()

In [ ]:
#import functions
from global_functions import *

## Configurar e carregar imagens para treino dos modelos

In [ ]:
#Train only the emotions present in all datasets (training and testing)
emotions_labels = {0:'raiva',  1:'medo', 2:'alegria', 3: 'tristeza'}
class_names = ['raiva', 'medo', 'alegria', 'tristeza']

img_width = 224
img_height = 224
img_depth = 3
num_classes = len(class_names)
print('Classes:' + str(num_classes))

In [ ]:
#Replace value of DATASET variable for each available dataset
DATASET = 'NIMH'

## Train and test splits

In [ ]:
X_train = np.load('X_train_'+ DATASET + '.npy')
y_train = np.load('Y_train_' + DATASET + '.npy') 

X_valid = np.load('X_valid_'+ DATASET + '.npy')
y_valid = np.load('Y_valid_' + DATASET + '.npy') 

X_test = np.load('X_test_'+ DATASET + '.npy')
y_test = np.load('Y_test_' + DATASET + '.npy') 

In [ ]:
#ImageDataGenerator to generate augmented images during model training 
batchSize = 20
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True,
                                   brightness_range=[0.5,1.5]
)

train_datagen.fit(X_train)

valid_datagen = ImageDataGenerator(rescale=1.0/255,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True,
                                   brightness_range=[0.5,1.5]
)

valid_datagen.fit(X_valid)

train_generator = train_datagen.flow(X_train, y_train, batch_size = batchSize)
valid_generator = train_datagen.flow(X_valid, y_valid, batch_size = batchSize)

## Calculate weights (class imbalance)

In [ ]:
from sklearn.utils import class_weight
classes_train = np.argmax(y_train, axis=1)
classes = np.unique(classes_train)
weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(classes_train), y=classes_train)
weights = np.round(weights,6)
class_weights = {k: v for k, v in zip(classes, weights)}
print('Class weights:', class_weights)

## Create models and train with added classes

### VGG16 - M1

In [ ]:
model = createModel('VGG16', img_height, img_width, modelType='m1', num_classes=4)

In [ ]:
 modelTrained = compileAndTrain('VGG16', 'm1', DATASET, model, train_generator, valid_generator, 
                                X_train, class_weights) 

In [ ]:
yhat = test_model('VGG16', 'm1', DATASET, modelTrained, X_test, y_test, emotions_labels)

### VGG16 - M2

In [ ]:
model = createModel('VGG16', img_height, img_width, modelType='m2', num_classes=4)

In [ ]:
 modelTrained = compileAndTrain('VGG16', 'm2', DATASET, model, train_generator, valid_generator, 
                                X_train, class_weights) 

In [ ]:
yhat = test_model('VGG16', 'm2', DATASET, modelTrained, X_test, y_test, emotions_labels)